Machine Learning Coursework on Loan Prediction

LOAD NECESSARY PYTHON LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import warnings # warning filter

#plotting libraries
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

#relevant ML libraries
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


LOAD OUR LOAN TRAINING AND TEST DATASET

In [ ]:
train = pd.read_csv("train_u6lujuX_CVtuZ9i.csv")
test = pd.read_csv("test_Y3wMUE5_7gLdaTN.csv")
ss = pd.read_csv("sample_submission.csv")

TO DISPLAY TRAIN DATA SIZE

In [ ]:
train.shape

TO DISPLAY TEST DATA SIZE

In [ ]:
test.shape

TO SHOW OUR TEST DATA

In [ ]:
test

TO DO A DATA PREPROCESSING - TO DROP THE UNWANTED COLUMN (LOAN ID)

In [ ]:
data = pd.concat([train, test])
data.drop("Loan_ID", axis=1, inplace=True)
data

TO IDENTIFY MISSING VALUES

In [ ]:
data.isnull().sum()

INPUT THE MISSING VALUES

In [ ]:
for i in [data]:
    i["Gender"] = i["Gender"].fillna(data.Gender.dropna().mode()[0])
    i["Married"] = i["Married"].fillna(data.Married.dropna().mode()[0])
    i["Dependents"] = i["Dependents"].fillna(data.Dependents.dropna().mode()[0])
    i["Self_Employed"] = i["Self_Employed"].fillna(data.Self_Employed.dropna().mode()[0])
    i["LoanAmount"] = i["LoanAmount"].fillna(data.LoanAmount.dropna().mode()[0])
    i["Loan_Amount_Term"] = i["Loan_Amount_Term"].fillna(data.Loan_Amount_Term.dropna().mode()[0])
    i["Credit_History"] = i["Credit_History"].fillna(data.Credit_History.dropna().mode()[0])
    i["Loan_Status"] = i["Loan_Status"].fillna(data.Loan_Status.dropna().mode()[0])


In [ ]:
# import warnings
# warnings.filterwarnings(action='ignore', category=DeprecationWarning)
# warnings.filterwarnings(action='ignore', category=UserWarning)
# warnings.filterwarnings(action='ignore', category=FutureWarning)
# warnings.filterwarnings(action='ignore', category=RuntimeWarning)

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


from sklearn.ensemble import RandomForestRegressor
data1 = data.loc[:, ['LoanAmount', 'Loan_Amount_Term']]

imp = IterativeImputer(RandomForestRegressor(), max_iter=10, random_state=0)
data1 = pd.DataFrame(imp.fit_transform(data1), columns=data1.columns)

In [ ]:
for i in [data]:
    i["Gender"] = i["Gender"].map({"Male":0,"Female":1}).astype(int)
    i["Married"] = i["Married"].map({"No":0,"Yes":1}).astype(int)
    i["Education"] = i["Education"].map({"Not Graduate":0,"Graduate":1}).astype(int)
    i["Self_Employed"] = i["Self_Employed"].map({"No":0,"Yes":1}).astype(int)
    i["Credit_History"] = i["Credit_History"].astype(int)
   

In [ ]:
new_train = data.iloc[:614]
new_test = data.iloc[614:]

# pd.set_option('mode.chained_assignment',None)


In [ ]:
new_train["Loan_Status"] = new_train["Loan_Status"].map({"N":0,"Y":1}).astype(int)

UNIVARIATE ANALYSIS

In [ ]:
fig,ax = plt.subplots(2,4,figsize=(16,10))
sns.countplot('Loan_Status',data=new_train,ax=ax[0][0])
sns.countplot('Gender',data=new_train,ax=ax[0][1])
sns.countplot('Married',data=new_train,ax=ax[0][2])
sns.countplot('Education',data=new_train,ax=ax[0][3])
sns.countplot('Self_Employed',data=new_train,ax=ax[1][0])
sns.countplot('Property_Area',data=new_train,ax=ax[1][1])
sns.countplot('Credit_History',data=new_train,ax=ax[1][2])
sns.countplot('Dependents',data=new_train,ax=ax[1][3])

BIVARIATE ANALYSIS

In [ ]:
sns.boxplot(x='Loan_Status',y='ApplicantIncome',data=new_train)

In [ ]:
sns.boxplot(x='Loan_Status',y='CoapplicantIncome',data=new_train)

In [ ]:
sns.catplot(x='Gender',y='LoanAmount',data=new_train,kind='box')

In [ ]:
sns.catplot(x='Gender',y='LoanAmount',data=data,kind='box',hue='Loan_Status', col='Married')

In [ ]:
sns.catplot(x='Gender',y='CoapplicantIncome',data=data,kind='boxen',hue='Loan_Status', col='Property_Area')

In [ ]:
# r = 0.00833
# data['EMI'] = data.apply(lambda x: (x['LoanAmount']*r*((1+r)**x['Loan_Amount_Term']))/((1+r)**((x['Loan_Amount_Term'])-1)), axis=1)
# data['Dependents_EMI_mean'] = data.groupby(['Dependents'])['EMI'].transform('mean')
# data['LoanAmount_per_TotalIncome'] = data['LoanAmount']/data['TotalIncome']
# data['Loan_Amount_Term_per_TotalIncome'] = data['Loan_Amount_Term']/data['TotalIncome']
# data['EMI_per_Loan_Amount_Term'] = data['EMI']/data['Loan_Amount_Term']
# data['EMI_per_LoanAmount'] = data['EMI']/data['LoanAmount']
# data['Property_Area_LoanAmount_per_TotalIncome_mean'] = data.groupby(['Property_Area'])['LoanAmount_per_TotalIncome'].transform('mean')
# data['Credit_History_Income_Sum'] = data.groupby(['Credit_History'])['TotalIncome'].transform('sum')
# data['Dependents_LoanAmount_Sum'] = data.groupby(['Dependents'])['LoanAmount'].transform('sum')

plt.figure(figsize = (10,10))
correlation_martrix = new_train.corr()
sns.heatmap(correlation_martrix,annot=True)
plt.show

GET TOTAL INCOME

In [ ]:
for i in [data]:
    i["TotalIncome"] = i["ApplicantIncome"]+i["CoapplicantIncome"]

i["TotalIncome"]

GET EQUATED MONTHLY INSTALLMENTS

In [ ]:
r = 0.00833
data['EMI'] = data.apply(lambda x: (x['LoanAmount']*r*((1+r)**x['Loan_Amount_Term']))/((1+r)**((x['Loan_Amount_Term'])-1)), axis=1)

ADDITIONAL FEATURES

In [ ]:
data['Dependents_EMI_mean'] = data.groupby(['Dependents'])['EMI'].transform('mean')

data['LoanAmount_per_TotalIncome'] = data['LoanAmount']/data['TotalIncome']

data['Loan_Amount_Term_per_TotalIncome'] = data['Loan_Amount_Term']/data['TotalIncome']

data['EMI_per_Loan_Amount_Term'] = data['EMI']/data['Loan_Amount_Term']

data['EMI_per_LoanAmount'] = data['EMI']/data['LoanAmount']

data['Property_Area_LoanAmount_per_TotalIncome_mean'] = data.groupby(['Property_Area'])['LoanAmount_per_TotalIncome'].transform('mean')

data['Credit_History_Income_Sum'] = data.groupby(['Credit_History'])['TotalIncome'].transform('sum')

data['Dependents_LoanAmount_Sum'] = data.groupby(['Dependents'])['LoanAmount'].transform('sum')


BIN INFORMATION

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

Loan_Amount_Term_discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
data['Loan_Amount_Term_Bins'] = Loan_Amount_Term_discretizer.fit_transform(data['Loan_Amount_Term'].values.reshape(-1,1)).astype(float)

TotalIncome_discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
data['TotalIncome_Bins'] = TotalIncome_discretizer.fit_transform(data['TotalIncome'].values.reshape(-1,1)).astype(float)

LoanAmount_per_TotalIncome_discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
data['LoanAmount_per_TotalIncome_Bins'] = LoanAmount_per_TotalIncome_discretizer.fit_transform(data['LoanAmount_per_TotalIncome'].values.reshape(-1,1)).astype(float)

REMOVE UNWANTED COLUMN

In [ ]:
data=data.drop(['EMI'],axis=1)
data=data.drop(['TotalIncome'],axis=1)
data=data.drop(['LoanAmount_per_TotalIncome'],axis=1)

SIZE AFTER UNWANTED FEATURES

In [ ]:
new_train.shape

ADDING MODEL

In [ ]:
x = new_train.drop("Loan_Status",axis=1)
y = new_train["Loan_Status"]

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)


In [ ]:
x_train.shape

In [ ]:
x_test.shape

ALGORITHM FOR TRAINING

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer

cross_val_score(model,x_train,y_train,scoring=make_scorer(accuracy_score),cv=3)

In [ ]:
pred = model.fit(x_train,y_train).predict(x_test)
accuracy_score(pred,y_test)

Using GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
LR_parameter_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'max_iter': list(range(100,800,100)),
    'solver': ['netwon-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

LR_search = GridSearchCV(LogisticRegression(), LR_parameter_grid, refit=True, verbose=3, cv=5)
LR_search.fit(x_train, y_train)
LR_search.best_params_

print('Mean Accuracy: %.3f' % LR_search.best_score_)
print('Config: %s' % LR_search.best_params_)

In [ ]:
ll = LR_search.predict(x_test)
accuracy_score(ll, y_test)

TEST DATA PREDICTION

In [ ]:
hhh = LR_search.predict(new_test)

PREPARE THE SUBMISSION FILE 

In [ ]:
dftest = pd.DataFrame(data=hhh, columns=["Loan_Status"])
finalpred = pd.concat([ss["Loan_ID"], dftest],axis=1)
finalpred["Loan_Status"]=finalpred["Loan_Status"].map({1:'Y',0:'N'})
finalpred.to_csv("finalprediction.csv", index=False)